# 08 - Tools and Agents: Building Autonomous AI Systems

## Overview
In this notebook, we'll learn how to create autonomous AI agents that can use tools to accomplish complex tasks. You'll build agents that can search the web, perform calculations, interact with APIs, and make decisions independently.

## Learning Objectives
By the end of this notebook, you will be able to:
- Create custom tools for agents to use
- Build ReAct agents that reason and act
- Implement function calling with OpenAI models
- Design multi-step agent workflows
- Create specialized agents for different tasks
- Handle agent errors and implement retry logic

## Prerequisites
- Completion of notebooks 01-07
- Understanding of LLMs and prompt engineering
- Basic knowledge of Python functions and classes

## Back-and-Forth Teaching Pattern
This notebook follows our pattern:
1. **Instructor Activity**: Demonstrates a concept with complete examples
2. **Learner Activity**: You apply the concept with guidance and hidden solutions

## Setup

Let's install and import the necessary libraries:

In [ ]:
# Install required packages
!pip install langchain langchain-community langchain-openai langchain-experimental wikipedia-api python-dotenv

In [ ]:
import os
from typing import Optional, Type, List, Dict, Any
from langchain.agents import AgentExecutor, create_react_agent, Tool
from langchain.agents import create_openai_functions_agent
from langchain.agents import create_structured_chat_agent
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.callbacks.base import BaseCallbackHandler
from pydantic import BaseModel, Field
import json
import math
import warnings
warnings.filterwarnings('ignore')

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

---

## Instructor Activity 1: Creating Tools and Basic Agents

Let's start by creating custom tools and building our first agent:

In [ ]:
# Method 1: Create tools using the @tool decorator
@tool
def calculate(expression: str) -> str:
    """Evaluate a mathematical expression. 
    
    Args:
        expression: A mathematical expression like '2 + 2' or 'sqrt(16)'
    """
    try:
        # Create safe namespace for eval
        safe_dict = {
            'sqrt': math.sqrt,
            'pow': math.pow,
            'sin': math.sin,
            'cos': math.cos,
            'tan': math.tan,
            'pi': math.pi,
            'e': math.e
        }
        result = eval(expression, {"__builtins__": {}}, safe_dict)
        return f"The result is: {result}"
    except Exception as e:
        return f"Error calculating: {str(e)}"

@tool
def get_word_count(text: str) -> str:
    """Count the number of words in a text.
    
    Args:
        text: The text to count words in
    """
    word_count = len(text.split())
    return f"The text contains {word_count} words."

# Test the tools
print(calculate.invoke({"expression": "sqrt(16) + 5"}))
print(get_word_count.invoke({"text": "Hello world, how are you today?"}))

In [ ]:
# Method 2: Create structured tools with input validation

class SearchInput(BaseModel):
    query: str = Field(description="The search query")
    max_results: int = Field(default=3, description="Maximum number of results")

def search_wikipedia(query: str, max_results: int = 3) -> str:
    """Search Wikipedia for information."""
    try:
        wiki = WikipediaAPIWrapper()
        wiki.top_k_results = max_results
        results = wiki.run(query)
        return results[:500]  # Limit response length
    except Exception as e:
        return f"Search error: {str(e)}"

# Create structured tool
wikipedia_tool = StructuredTool.from_function(
    func=search_wikipedia,
    name="wikipedia_search",
    description="Search Wikipedia for information on any topic",
    args_schema=SearchInput
)

# Test the tool
result = wikipedia_tool.invoke({"query": "artificial intelligence", "max_results": 1})
print(result[:200] + "...")

In [ ]:
# Method 3: Create custom tool class for complex logic

class WeatherTool(BaseTool):
    name = "get_weather"
    description = "Get current weather for a city (simulated data for demo)"
    
    def _run(self, city: str) -> str:
        """Execute the tool."""
        # Simulated weather data (in production, call real API)
        weather_data = {
            "New York": {"temp": 72, "condition": "Sunny", "humidity": 65},
            "London": {"temp": 59, "condition": "Cloudy", "humidity": 80},
            "Tokyo": {"temp": 68, "condition": "Partly cloudy", "humidity": 70},
            "Sydney": {"temp": 77, "condition": "Clear", "humidity": 55}
        }
        
        city_key = city.title()
        if city_key in weather_data:
            data = weather_data[city_key]
            return f"Weather in {city_key}: {data['temp']}°F, {data['condition']}, Humidity: {data['humidity']}%"
        else:
            return f"Weather data not available for {city}. Try New York, London, Tokyo, or Sydney."
    
    async def _arun(self, city: str) -> str:
        """Async version (optional)."""
        return self._run(city)

# Create instance
weather_tool = WeatherTool()

# Test
print(weather_tool.run("New York"))

In [ ]:
# Create a ReAct agent with our tools
from langchain import hub

# Create LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Gather tools
tools = [calculate, get_word_count, wikipedia_tool, weather_tool]

# Get ReAct prompt (or create custom one)
prompt = hub.pull("hwchase17/react")

# Create agent
agent = create_react_agent(llm, tools, prompt)

# Create agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # Show reasoning steps
    handle_parsing_errors=True,
    max_iterations=5  # Limit iterations to prevent infinite loops
)

# Test the agent
result = agent_executor.invoke({
    "input": "What's the weather in Tokyo? Also calculate the temperature in Celsius using the formula (F-32)*5/9"
})

print("\nFinal Answer:", result["output"])

In [ ]:
# Create agent with custom reasoning prompt

custom_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant with access to tools.
    
    To use a tool, follow this format exactly:
    Thought: I need to [explain your reasoning]
    Action: tool_name
    Action Input: the input to the tool
    
    After receiving the tool result:
    Observation: [tool result will appear here]
    
    Continue this process until you have enough information.
    
    When ready to give the final answer:
    Thought: I now have all the information needed
    Final Answer: [your complete answer]
    
    Available tools:
    {tools}
    
    Tool Names: {tool_names}"""),
    ("human", "{input}\n{agent_scratchpad}")
])

# Create agent with custom prompt
custom_agent = create_react_agent(llm, tools, custom_prompt)

custom_executor = AgentExecutor(
    agent=custom_agent,
    tools=tools,
    verbose=True,
    max_iterations=4
)

# Complex multi-step task
result = custom_executor.invoke({
    "input": """Find information about the Pythagorean theorem on Wikipedia. 
    Then use it to calculate the hypotenuse of a right triangle with sides 3 and 4."""
})

print("\nFinal Result:", result["output"])

---

## Learner Activity 1: Build Your Own Agent with Custom Tools

Create an agent that can help with research and data analysis tasks.

**Task**: Build an agent with custom tools for:
1. Text analysis (sentiment, summary, key points extraction)
2. Data manipulation (sorting, filtering, basic statistics)
3. Note-taking (save and retrieve notes)
4. Task planning (break down complex tasks)

Requirements:
- Create at least 4 custom tools
- Build an agent that can chain tools together
- Handle errors gracefully
- Include examples of multi-step reasoning

In [ ]:
# Create your custom tools

# TODO: Tool 1 - Text sentiment analysis
@tool
def analyze_sentiment(text: str) -> str:
    """Analyze the sentiment of text (positive, negative, neutral)."""
    # Your implementation here
    pass

# TODO: Tool 2 - Extract key points from text
@tool
def extract_key_points(text: str, num_points: int = 3) -> str:
    """Extract key points from text."""
    # Your implementation here
    pass

# TODO: Tool 3 - Calculate statistics
@tool
def calculate_statistics(numbers_str: str) -> str:
    """Calculate mean, median, min, max from comma-separated numbers."""
    # Parse numbers and calculate stats
    # Your implementation here
    pass

# TODO: Tool 4 - Note management
class NoteManager:
    # Create a class to manage notes
    # Should support save_note and get_notes
    pass

# TODO: Tool 5 - Task planner
@tool
def create_task_plan(goal: str) -> str:
    """Break down a goal into actionable steps."""
    # Your implementation here
    pass

# TODO: Create your research agent
# Combine all tools into an agent
# Test with complex research tasks

# Your code here

In [ ]:
# Solution (hidden by default)

"""
# Custom research and analysis tools

@tool
def analyze_sentiment(text: str) -> str:
    '''Analyze the sentiment of text (positive, negative, neutral).'''
    # Simple keyword-based sentiment (in production, use proper NLP)
    positive_words = ['good', 'great', 'excellent', 'amazing', 'wonderful', 'fantastic', 'love']
    negative_words = ['bad', 'terrible', 'awful', 'horrible', 'hate', 'poor', 'worst']
    
    text_lower = text.lower()
    positive_count = sum(1 for word in positive_words if word in text_lower)
    negative_count = sum(1 for word in negative_words if word in text_lower)
    
    if positive_count > negative_count:
        sentiment = "POSITIVE"
        confidence = positive_count / (positive_count + negative_count + 1)
    elif negative_count > positive_count:
        sentiment = "NEGATIVE"
        confidence = negative_count / (positive_count + negative_count + 1)
    else:
        sentiment = "NEUTRAL"
        confidence = 0.5
    
    return f"Sentiment: {sentiment} (Confidence: {confidence:.2%})"

@tool
def extract_key_points(text: str, num_points: str = "3") -> str:
    '''Extract key points from text.'''
    try:
        num = int(num_points)
    except:
        num = 3
    
    # Split into sentences
    sentences = text.replace('!', '.').replace('?', '.').split('.')
    sentences = [s.strip() for s in sentences if len(s.strip()) > 10]
    
    if not sentences:
        return "No key points found in text."
    
    # Simple extraction: take first/important sentences
    key_points = sentences[:num] if len(sentences) >= num else sentences
    
    result = "Key Points:\n"
    for i, point in enumerate(key_points, 1):
        result += f"{i}. {point}\n"
    
    return result

@tool
def calculate_statistics(numbers_str: str) -> str:
    '''Calculate mean, median, min, max from comma-separated numbers.'''
    try:
        numbers = [float(n.strip()) for n in numbers_str.split(',')]
        
        if not numbers:
            return "No valid numbers provided."
        
        import statistics
        
        stats = {
            "Count": len(numbers),
            "Mean": round(statistics.mean(numbers), 2),
            "Median": round(statistics.median(numbers), 2),
            "Min": round(min(numbers), 2),
            "Max": round(max(numbers), 2),
            "Std Dev": round(statistics.stdev(numbers), 2) if len(numbers) > 1 else 0
        }
        
        result = "Statistics:\n"
        for key, value in stats.items():
            result += f"- {key}: {value}\n"
        
        return result
    except Exception as e:
        return f"Error calculating statistics: {str(e)}"

# Note management system
class NoteManager:
    def __init__(self):
        self.notes = {}
    
    def save_note(self, title: str, content: str) -> str:
        self.notes[title] = {
            "content": content,
            "timestamp": str(datetime.now())
        }
        return f"Note '{title}' saved successfully."
    
    def get_notes(self, title: str = None) -> str:
        if title:
            if title in self.notes:
                note = self.notes[title]
                return f"Note: {title}\nContent: {note['content']}\nSaved: {note['timestamp']}"
            else:
                return f"Note '{title}' not found."
        else:
            if self.notes:
                return "Available notes: " + ", ".join(self.notes.keys())
            else:
                return "No notes saved yet."

# Create note manager instance
note_manager = NoteManager()

# Create note tools
@tool
def save_note(title: str, content: str) -> str:
    '''Save a note with a title and content.'''
    return note_manager.save_note(title, content)

@tool
def get_notes(title: str = "") -> str:
    '''Get saved notes. Provide title for specific note, or leave empty for list.'''
    return note_manager.get_notes(title if title else None)

@tool
def create_task_plan(goal: str) -> str:
    '''Break down a goal into actionable steps.'''
    # Simple task decomposition
    templates = {
        "research": [
            "1. Define the research question clearly",
            "2. Gather relevant sources and materials",
            "3. Analyze and synthesize information",
            "4. Draw conclusions",
            "5. Document findings"
        ],
        "analysis": [
            "1. Collect and prepare data",
            "2. Perform exploratory analysis",
            "3. Apply analytical methods",
            "4. Interpret results",
            "5. Create summary report"
        ],
        "default": [
            "1. Clarify the objective",
            "2. Break down into sub-tasks",
            "3. Prioritize tasks",
            "4. Execute step by step",
            "5. Review and refine"
        ]
    }
    
    # Detect task type
    goal_lower = goal.lower()
    if "research" in goal_lower or "find" in goal_lower:
        plan = templates["research"]
    elif "analyz" in goal_lower or "data" in goal_lower:
        plan = templates["analysis"]
    else:
        plan = templates["default"]
    
    result = f"Task Plan for: {goal}\n\n"
    result += "\n".join(plan)
    result += "\n\nNote: Adapt these steps based on your specific needs."
    
    return result

# Create the research agent
from datetime import datetime

research_tools = [
    analyze_sentiment,
    extract_key_points,
    calculate_statistics,
    save_note,
    get_notes,
    create_task_plan,
    wikipedia_tool  # Include Wikipedia for research
]

# Create agent
research_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

research_prompt = ChatPromptTemplate.from_messages([
    ("system", '''You are a research assistant with analytical tools.
    
    Approach tasks systematically:
    1. Understand the request
    2. Plan your approach using available tools
    3. Execute step by step
    4. Save important findings as notes
    5. Provide comprehensive analysis
    
    Format:
    Thought: [your reasoning]
    Action: [tool name]
    Action Input: [tool input]
    Observation: [tool output]
    
    Final Answer: [comprehensive response]
    
    Tools: {tools}
    Tool Names: {tool_names}'''),
    ("human", "{input}\n{agent_scratchpad}")
])

research_agent = create_react_agent(research_llm, research_tools, research_prompt)

research_executor = AgentExecutor(
    agent=research_agent,
    tools=research_tools,
    verbose=True,
    max_iterations=6,
    handle_parsing_errors=True
)

# Test the research agent
print("Testing Research Agent:\n")

# Test 1: Research and analysis
result1 = research_executor.invoke({
    "input": '''Research artificial intelligence on Wikipedia, 
    extract 3 key points, analyze the sentiment, and save findings as a note titled "AI Research".'''
})
print("\nResult 1:", result1["output"])

print("\n" + "="*60 + "\n")

# Test 2: Data analysis
result2 = research_executor.invoke({
    "input": '''Calculate statistics for these test scores: 85, 92, 78, 95, 88, 91, 76, 89. 
    Then create a task plan for improving student performance.'''
})
print("\nResult 2:", result2["output"])

print("\n" + "="*60 + "\n")

# Test 3: Retrieve saved information
result3 = research_executor.invoke({
    "input": "Show me all saved notes and retrieve the AI Research note."
})
print("\nResult 3:", result3["output"])
"""

print("Create a research agent with custom analytical tools!")
print("The solution shows text analysis, statistics, note-taking, and planning capabilities.")

---

## Instructor Activity 2: OpenAI Function Calling and Advanced Agents

Let's explore OpenAI's function calling capability and build more sophisticated agents:

In [ ]:
# OpenAI Function Calling Agent
from langchain.agents import create_openai_functions_agent
from langchain.schema import SystemMessage

# Create tools with proper descriptions for function calling
@tool
def get_stock_price(symbol: str) -> str:
    """Get the current stock price for a symbol (simulated data).
    
    Args:
        symbol: Stock ticker symbol (e.g., AAPL, GOOGL)
    """
    # Simulated stock prices
    stocks = {
        "AAPL": 178.25,
        "GOOGL": 138.92,
        "MSFT": 378.85,
        "AMZN": 145.73,
        "TSLA": 242.64
    }
    
    symbol_upper = symbol.upper()
    if symbol_upper in stocks:
        return f"${stocks[symbol_upper]} per share"
    return f"Stock price not found for {symbol}"

@tool
def calculate_portfolio_value(holdings: str) -> str:
    """Calculate total portfolio value from holdings.
    
    Args:
        holdings: Comma-separated list of 'symbol:shares' (e.g., 'AAPL:10,GOOGL:5')
    """
    stocks = {
        "AAPL": 178.25,
        "GOOGL": 138.92,
        "MSFT": 378.85,
        "AMZN": 145.73,
        "TSLA": 242.64
    }
    
    total = 0
    details = []
    
    for holding in holdings.split(','):
        try:
            symbol, shares = holding.strip().split(':')
            symbol = symbol.upper()
            shares = int(shares)
            
            if symbol in stocks:
                value = stocks[symbol] * shares
                total += value
                details.append(f"{symbol}: {shares} shares × ${stocks[symbol]} = ${value:,.2f}")
        except:
            continue
    
    if details:
        return "\n".join(details) + f"\n\nTotal Portfolio Value: ${total:,.2f}"
    return "No valid holdings found"

# Tools for function calling
function_tools = [get_stock_price, calculate_portfolio_value, calculate]

# Create function calling agent
function_llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0)

function_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful financial assistant that can check stock prices and analyze portfolios."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

function_agent = create_openai_functions_agent(
    llm=function_llm,
    tools=function_tools,
    prompt=function_prompt
)

function_executor = AgentExecutor(
    agent=function_agent,
    tools=function_tools,
    verbose=True
)

# Test function calling
result = function_executor.invoke({
    "input": "What's the current price of Apple stock? Also calculate the value of a portfolio with 10 AAPL and 5 GOOGL shares."
})

print("\nResult:", result["output"])

In [ ]:
# Agent with conversation memory
from langchain.memory import ConversationBufferMemory

# Create memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Agent with memory prompt
memory_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="""You are a helpful assistant with memory of our conversation.
    Use your tools when needed and remember what we discussed."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Create agent with memory
memory_agent = create_openai_functions_agent(
    llm=function_llm,
    tools=function_tools,
    prompt=memory_prompt
)

memory_executor = AgentExecutor(
    agent=memory_agent,
    tools=function_tools,
    memory=memory,
    verbose=True
)

# Have a conversation
print("Conversation with memory:\n")

response1 = memory_executor.invoke({"input": "Check the price of Microsoft stock."})
print("Response 1:", response1["output"])

print("\n" + "="*50 + "\n")

response2 = memory_executor.invoke({"input": "What stock did I just ask about?"})
print("Response 2:", response2["output"])

print("\n" + "="*50 + "\n")

response3 = memory_executor.invoke({
    "input": "Calculate how much 20 shares of that stock would cost."
})
print("Response 3:", response3["output"])

In [ ]:
# Agent with error handling and retry logic

class RobustAgent:
    def __init__(self, executor, max_retries=3):
        self.executor = executor
        self.max_retries = max_retries
        self.error_log = []
    
    def invoke(self, input_dict: Dict) -> Dict:
        """Execute with retry logic and error handling."""
        retries = 0
        last_error = None
        
        while retries < self.max_retries:
            try:
                # Attempt execution
                result = self.executor.invoke(input_dict)
                
                # Check for parsing errors in output
                if "error" not in result.get("output", "").lower():
                    return result
                else:
                    # Refine input for retry
                    input_dict["input"] = f"Please try again. {input_dict['input']}"
                    retries += 1
                    
            except Exception as e:
                last_error = str(e)
                self.error_log.append({
                    "attempt": retries + 1,
                    "error": last_error,
                    "input": input_dict["input"]
                })
                
                retries += 1
                
                if retries < self.max_retries:
                    print(f"Retry {retries}/{self.max_retries} after error: {last_error[:100]}")
                    # Add context about error to help agent
                    input_dict["input"] = f"{input_dict['input']} (Previous attempt failed, please be careful)"
        
        # All retries exhausted
        return {
            "output": f"Failed after {self.max_retries} attempts. Last error: {last_error}",
            "error": True
        }
    
    def get_error_summary(self) -> str:
        """Get summary of errors."""
        if not self.error_log:
            return "No errors logged."
        
        summary = f"Total errors: {len(self.error_log)}\n"
        for error in self.error_log[-3:]:  # Last 3 errors
            summary += f"- Attempt {error['attempt']}: {error['error'][:100]}\n"
        
        return summary

# Create robust agent
robust_agent = RobustAgent(function_executor)

# Test with potentially problematic input
test_cases = [
    "Calculate the square root of negative one",  # Math error
    "What's the stock price of XYZ?",  # Unknown stock
    "Calculate portfolio value for AAPL:ten",  # Parse error
]

for test in test_cases:
    print(f"\nTesting: {test}")
    result = robust_agent.invoke({"input": test})
    print(f"Result: {result['output']}")

print("\n" + "="*50)
print("\nError Summary:")
print(robust_agent.get_error_summary())

---

## Learner Activity 2: Build a Multi-Tool Assistant Agent

Create a sophisticated assistant that can handle various tasks with function calling.

**Task**: Build an assistant that can:
1. Manage a to-do list (add, complete, list tasks)
2. Set and check reminders
3. Answer general knowledge questions
4. Perform calculations and conversions
5. Generate creative content (stories, jokes)

Requirements:
- Use OpenAI function calling
- Implement conversation memory
- Add error recovery
- Create at least 5 specialized tools
- Handle multi-step requests

In [ ]:
# Build your multi-tool assistant

# TODO: Create a TodoManager class
class TodoManager:
    def __init__(self):
        self.tasks = []
    
    # Add methods for add_task, complete_task, list_tasks
    # Your implementation here
    pass

# TODO: Create todo management tools
@tool
def add_task(description: str, priority: str = "medium") -> str:
    """Add a new task to the todo list."""
    # Your implementation here
    pass

@tool
def complete_task(task_id: int) -> str:
    """Mark a task as complete."""
    # Your implementation here
    pass

# TODO: Create reminder tools
@tool
def set_reminder(message: str, time: str) -> str:
    """Set a reminder for later."""
    # Your implementation here
    pass

# TODO: Create conversion tool
@tool
def convert_units(value: float, from_unit: str, to_unit: str) -> str:
    """Convert between different units."""
    # Your implementation here
    pass

# TODO: Create creative content generator
@tool
def generate_creative_content(content_type: str, topic: str) -> str:
    """Generate creative content like stories or jokes."""
    # Your implementation here
    pass

# TODO: Build the assistant agent
# - Combine all tools
# - Add memory
# - Create helpful system prompt
# - Test with complex requests

# Your code here

In [ ]:
# Solution (hidden by default)

"""
from datetime import datetime, timedelta
import random

# Todo Manager
class TodoManager:
    def __init__(self):
        self.tasks = []
        self.next_id = 1
    
    def add_task(self, description: str, priority: str = "medium") -> Dict:
        task = {
            "id": self.next_id,
            "description": description,
            "priority": priority,
            "status": "pending",
            "created": datetime.now().isoformat()
        }
        self.tasks.append(task)
        self.next_id += 1
        return task
    
    def complete_task(self, task_id: int) -> bool:
        for task in self.tasks:
            if task["id"] == task_id:
                task["status"] = "completed"
                task["completed"] = datetime.now().isoformat()
                return True
        return False
    
    def list_tasks(self, status: str = "all") -> List[Dict]:
        if status == "all":
            return self.tasks
        return [t for t in self.tasks if t["status"] == status]

# Reminder Manager
class ReminderManager:
    def __init__(self):
        self.reminders = []
    
    def add_reminder(self, message: str, time_str: str) -> Dict:
        reminder = {
            "message": message,
            "time": time_str,
            "created": datetime.now().isoformat()
        }
        self.reminders.append(reminder)
        return reminder
    
    def get_reminders(self) -> List[Dict]:
        return self.reminders

# Create managers
todo_mgr = TodoManager()
reminder_mgr = ReminderManager()

# Todo tools
@tool
def add_task(description: str, priority: str = "medium") -> str:
    '''Add a new task to the todo list.
    Priority can be: high, medium, or low.'''
    if priority not in ["high", "medium", "low"]:
        priority = "medium"
    
    task = todo_mgr.add_task(description, priority)
    return f"Task added: [{task['id']}] {task['description']} (Priority: {task['priority']})"

@tool
def complete_task(task_id: str) -> str:
    '''Mark a task as complete by its ID.'''
    try:
        id_num = int(task_id)
        if todo_mgr.complete_task(id_num):
            return f"Task {id_num} marked as complete!"
        return f"Task {id_num} not found."
    except:
        return "Invalid task ID. Please provide a number."

@tool
def list_tasks(status: str = "pending") -> str:
    '''List tasks. Status can be: pending, completed, or all.'''
    tasks = todo_mgr.list_tasks(status)
    
    if not tasks:
        return f"No {status} tasks found."
    
    result = f"{status.capitalize()} Tasks:\n"
    for task in tasks:
        emoji = "✅" if task["status"] == "completed" else "📝"
        priority_emoji = {"high": "🔴", "medium": "🟡", "low": "🟢"}.get(task["priority"], "")
        result += f"{emoji} [{task['id']}] {task['description']} {priority_emoji}\n"
    
    return result

# Reminder tools
@tool
def set_reminder(message: str, time_description: str = "in 1 hour") -> str:
    '''Set a reminder with a message and time.'''
    reminder = reminder_mgr.add_reminder(message, time_description)
    return f"Reminder set: '{message}' for {time_description}"

@tool
def check_reminders() -> str:
    '''Check all active reminders.'''
    reminders = reminder_mgr.get_reminders()
    
    if not reminders:
        return "No reminders set."
    
    result = "Active Reminders:\n"
    for i, reminder in enumerate(reminders, 1):
        result += f"{i}. '{reminder['message']}' - {reminder['time']}\n"
    
    return result

# Unit conversion tool
@tool
def convert_units(value: str, from_unit: str, to_unit: str) -> str:
    '''Convert between different units of measurement.'''
    try:
        val = float(value)
    except:
        return "Invalid value. Please provide a number."
    
    # Conversion factors (simplified)
    conversions = {
        # Length
        ("meters", "feet"): 3.28084,
        ("feet", "meters"): 0.3048,
        ("miles", "kilometers"): 1.60934,
        ("kilometers", "miles"): 0.621371,
        # Temperature
        ("celsius", "fahrenheit"): lambda c: c * 9/5 + 32,
        ("fahrenheit", "celsius"): lambda f: (f - 32) * 5/9,
        # Weight
        ("pounds", "kilograms"): 0.453592,
        ("kilograms", "pounds"): 2.20462,
    }
    
    from_lower = from_unit.lower()
    to_lower = to_unit.lower()
    
    if (from_lower, to_lower) in conversions:
        factor = conversions[(from_lower, to_lower)]
        if callable(factor):
            result = factor(val)
        else:
            result = val * factor
        return f"{val} {from_unit} = {result:.2f} {to_unit}"
    
    return f"Conversion from {from_unit} to {to_unit} not supported."

# Creative content generator
@tool
def generate_creative_content(content_type: str, topic: str = "random") -> str:
    '''Generate creative content like stories, jokes, or poems.'''
    content_type_lower = content_type.lower()
    
    if "joke" in content_type_lower:
        jokes = [
            "Why do programmers prefer dark mode? Because light attracts bugs!",
            "Why did the AI go to therapy? It had too many deep issues!",
            "What's an AI's favorite snack? Computer chips!",
            "Why was the computer cold? It left its Windows open!"
        ]
        if "ai" in topic.lower() or "computer" in topic.lower():
            return random.choice(jokes)
        return f"Here's a joke: {random.choice(jokes)}"
    
    elif "story" in content_type_lower:
        return f'''Mini Story: "{topic.title()}"\n
        Once upon a time in a digital realm, there was a {topic}. 
        It had magical powers that could transform data into wisdom. 
        Every day, it helped people solve problems and create amazing things. 
        The end. (Want a longer story? Just ask!)'''
    
    elif "poem" in content_type_lower:
        return f'''A Haiku about {topic}:\n
        {topic.title()} so bright,
        Digital dreams take their flight,
        Future shining light.'''
    
    else:
        return "I can generate jokes, stories, or poems. What would you like?"

# Math and general knowledge tool
@tool
def answer_question(question: str) -> str:
    '''Answer general knowledge questions.'''
    # Simple Q&A (in production, use real knowledge base)
    qa_pairs = {
        "capital of france": "The capital of France is Paris.",
        "speed of light": "The speed of light is approximately 299,792,458 meters per second.",
        "largest planet": "Jupiter is the largest planet in our solar system.",
        "python creator": "Python was created by Guido van Rossum."
    }
    
    question_lower = question.lower()
    for key, answer in qa_pairs.items():
        if key in question_lower:
            return answer
    
    return "I'll need to search for that information. Try asking about capitals, planets, or scientific facts."

# Combine all tools
assistant_tools = [
    add_task,
    complete_task,
    list_tasks,
    set_reminder,
    check_reminders,
    convert_units,
    generate_creative_content,
    answer_question,
    calculate  # From earlier
]

# Create assistant with memory
from langchain.memory import ConversationBufferWindowMemory

assistant_memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    return_messages=True,
    k=5  # Remember last 5 exchanges
)

assistant_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content='''You are a helpful personal assistant with multiple capabilities.
    You can manage tasks, set reminders, answer questions, do calculations, convert units, and generate creative content.
    
    Be proactive and helpful. If someone mentions needing to do something, offer to add it as a task.
    Remember our conversation and refer back to it when relevant.
    Be friendly and occasionally add appropriate emojis.'''),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

assistant_llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0.5)

assistant_agent = create_openai_functions_agent(
    llm=assistant_llm,
    tools=assistant_tools,
    prompt=assistant_prompt
)

assistant_executor = AgentExecutor(
    agent=assistant_agent,
    tools=assistant_tools,
    memory=assistant_memory,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

# Test the assistant
print("Personal Assistant Demo:\n")

# Test 1: Task management
print("Test 1: Task Management")
r1 = assistant_executor.invoke({"input": "Add a task to buy groceries with high priority"})
print(r1["output"])

print("\n" + "="*60 + "\n")

# Test 2: Multi-step request
print("Test 2: Multi-step Request")
r2 = assistant_executor.invoke({
    "input": "Convert 100 fahrenheit to celsius, then tell me a joke about the weather"
})
print(r2["output"])

print("\n" + "="*60 + "\n")

# Test 3: Memory check
print("Test 3: Memory Check")
r3 = assistant_executor.invoke({"input": "What tasks do I have pending?"})
print(r3["output"])

print("\n" + "="*60 + "\n")

# Test 4: Complex request
print("Test 4: Complex Request")
r4 = assistant_executor.invoke({
    "input": "Set a reminder to complete task 1 in 2 hours, then write me a short poem about productivity"
})
print(r4["output"])
"""

print("Build a multi-tool personal assistant with memory and function calling!")
print("The solution includes task management, reminders, conversions, and creative content.")

---

## Instructor Activity 3: Multi-Agent Systems and Orchestration

Let's build systems where multiple agents work together:

In [ ]:
# Multi-Agent System with Role Specialization

class SpecializedAgent:
    """Base class for specialized agents."""
    
    def __init__(self, name: str, role: str, tools: List, llm=None):
        self.name = name
        self.role = role
        self.llm = llm or ChatOpenAI(temperature=0)
        self.tools = tools
        self.executor = self._create_executor()
    
    def _create_executor(self):
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=f"""You are {self.name}, a specialized agent.
            Your role: {self.role}
            Use your tools effectively to complete tasks in your domain."""),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        
        agent = create_openai_functions_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=prompt
        )
        
        return AgentExecutor(
            agent=agent,
            tools=self.tools,
            verbose=True
        )
    
    def run(self, task: str) -> str:
        """Execute a task."""
        result = self.executor.invoke({"input": task})
        return result["output"]

# Create specialized tools for different agents

# Research Agent Tools
@tool
def research_topic(topic: str) -> str:
    """Research a topic and return key findings."""
    return f"Research findings on {topic}: [Key concepts, recent developments, expert opinions]"

@tool
def find_sources(topic: str) -> str:
    """Find credible sources for a topic."""
    return f"Sources for {topic}: [Academic papers, Books, Verified websites]"

# Writer Agent Tools
@tool
def write_draft(topic: str, style: str = "professional") -> str:
    """Write a draft on a topic in specified style."""
    return f"Draft ({style}): Introduction to {topic}, main points, conclusion."

@tool
def edit_text(text: str, focus: str = "clarity") -> str:
    """Edit text with specific focus."""
    return f"Edited version focusing on {focus}: [Improved text]"

# Analyst Agent Tools
@tool
def analyze_data(data: str) -> str:
    """Analyze data and provide insights."""
    return "Analysis: [Patterns identified, Key metrics, Recommendations]"

@tool
def create_summary(content: str, length: str = "medium") -> str:
    """Create a summary of content."""
    return f"{length.capitalize()} summary: [Main points, Key takeaways]"

# Create specialized agents
research_agent = SpecializedAgent(
    name="ResearchBot",
    role="Research topics and find credible sources",
    tools=[research_topic, find_sources, wikipedia_tool]
)

writer_agent = SpecializedAgent(
    name="WriterBot",
    role="Create and edit written content",
    tools=[write_draft, edit_text]
)

analyst_agent = SpecializedAgent(
    name="AnalystBot",
    role="Analyze information and create summaries",
    tools=[analyze_data, create_summary]
)

In [ ]:
# Agent Orchestrator

class AgentOrchestrator:
    """Coordinates multiple specialized agents."""
    
    def __init__(self, agents: Dict[str, SpecializedAgent]):
        self.agents = agents
        self.workflow_history = []
        self.llm = ChatOpenAI(temperature=0)
    
    def plan_workflow(self, objective: str) -> List[Dict]:
        """Plan which agents to use and in what order."""
        planning_prompt = ChatPromptTemplate.from_template(
            """Given this objective: {objective}
            
            And these available agents:
            {agents}
            
            Create a workflow plan. Return a JSON list of steps:
            [{"agent": "agent_name", "task": "specific task", "depends_on": null or step_number}]
            
            Plan:"""
        )
        
        chain = planning_prompt | self.llm | StrOutputParser()
        
        agents_desc = "\n".join([
            f"- {name}: {agent.role}" 
            for name, agent in self.agents.items()
        ])
        
        plan_text = chain.invoke({
            "objective": objective,
            "agents": agents_desc
        })
        
        # Parse plan (simplified)
        # In production, properly parse JSON
        workflow = [
            {"agent": "research", "task": "Research the topic", "depends_on": None},
            {"agent": "writer", "task": "Write content based on research", "depends_on": 0},
            {"agent": "analyst", "task": "Analyze and summarize", "depends_on": 1}
        ]
        
        return workflow
    
    def execute_workflow(self, objective: str) -> Dict:
        """Execute a complete workflow."""
        print(f"\n🎯 Objective: {objective}\n")
        
        # Plan workflow
        workflow = self.plan_workflow(objective)
        print(f"📋 Workflow Plan: {len(workflow)} steps\n")
        
        results = {}
        
        for i, step in enumerate(workflow):
            print(f"\n📍 Step {i+1}: {step['agent']} agent")
            print(f"   Task: {step['task']}")
            
            # Get context from dependencies
            context = ""
            if step["depends_on"] is not None:
                prev_result = results.get(step["depends_on"], "")
                context = f"Previous result: {prev_result}\n\n"
            
            # Execute task
            agent_name = step["agent"]
            if agent_name in self.agents:
                agent = self.agents[agent_name]
                task_with_context = context + step["task"]
                result = agent.run(task_with_context)
                results[i] = result
                print(f"   ✅ Result: {result[:100]}...")
            else:
                print(f"   ❌ Agent {agent_name} not found")
        
        # Compile final result
        self.workflow_history.append({
            "objective": objective,
            "workflow": workflow,
            "results": results
        })
        
        return {
            "success": True,
            "objective": objective,
            "final_output": results.get(len(workflow)-1, "No output"),
            "all_results": results
        }

# Create orchestrator
orchestrator = AgentOrchestrator({
    "research": research_agent,
    "writer": writer_agent,
    "analyst": analyst_agent
})

# Test multi-agent workflow
result = orchestrator.execute_workflow(
    "Create a comprehensive report on artificial intelligence trends"
)

print("\n" + "="*60)
print("\n📊 Final Result:")
print(f"Success: {result['success']}")
print(f"Output: {result['final_output']}")

In [ ]:
# Agent Communication and Collaboration

class CollaborativeAgent(SpecializedAgent):
    """Agent that can communicate with other agents."""
    
    def __init__(self, name: str, role: str, tools: List, llm=None):
        super().__init__(name, role, tools, llm)
        self.inbox = []
        self.collaborators = {}
    
    def send_message(self, to_agent: str, message: str, request_type: str = "info"):
        """Send message to another agent."""
        if to_agent in self.collaborators:
            collaborator = self.collaborators[to_agent]
            collaborator.receive_message({
                "from": self.name,
                "message": message,
                "type": request_type,
                "timestamp": datetime.now().isoformat()
            })
            return f"Message sent to {to_agent}"
        return f"Agent {to_agent} not found"
    
    def receive_message(self, message: Dict):
        """Receive message from another agent."""
        self.inbox.append(message)
    
    def process_inbox(self) -> List[str]:
        """Process messages and respond."""
        responses = []
        
        for msg in self.inbox:
            if msg["type"] == "request":
                # Process request
                response = self.run(msg["message"])
                self.send_message(
                    msg["from"], 
                    f"Response to your request: {response}",
                    "response"
                )
                responses.append(f"Responded to {msg['from']}")
            elif msg["type"] == "info":
                responses.append(f"Received info from {msg['from']}")
        
        self.inbox = []  # Clear processed messages
        return responses
    
    def collaborate(self, task: str, with_agents: List[str]) -> str:
        """Collaborate with other agents on a task."""
        print(f"\n{self.name} initiating collaboration...")
        
        # Request help from collaborators
        for agent_name in with_agents:
            self.send_message(
                agent_name,
                f"Please help with: {task}",
                "request"
            )
        
        # Wait for responses (simplified)
        responses = []
        for agent_name in with_agents:
            if agent_name in self.collaborators:
                agent = self.collaborators[agent_name]
                agent_responses = agent.process_inbox()
                responses.extend(agent_responses)
        
        # Process responses and complete task
        own_result = self.run(task)
        
        return f"Collaborative result: {own_result}\nWith input from: {', '.join(with_agents)}"

# Create collaborative agents
collab_researcher = CollaborativeAgent(
    name="CollabResearcher",
    role="Research with collaboration",
    tools=[research_topic, find_sources]
)

collab_writer = CollaborativeAgent(
    name="CollabWriter",
    role="Write with input from others",
    tools=[write_draft, edit_text]
)

# Set up collaboration network
collab_researcher.collaborators = {"CollabWriter": collab_writer}
collab_writer.collaborators = {"CollabResearcher": collab_researcher}

# Test collaboration
result = collab_writer.collaborate(
    "Write an article about quantum computing",
    with_agents=["CollabResearcher"]
)

print(f"\nCollaboration Result: {result}")

---

## Learner Activity 3: Build a Multi-Agent Project Management System

Create a system where multiple agents collaborate to manage a software project.

**Task**: Build agents for:
1. **Project Manager**: Plans tasks and assigns to agents
2. **Developer**: Writes code and technical documentation
3. **Tester**: Creates test cases and reports bugs
4. **Reviewer**: Reviews code and documentation

Requirements:
- Each agent should have specialized tools
- Agents must communicate and collaborate
- Include a workflow orchestrator
- Handle task dependencies
- Track project progress
- Implement feedback loops between agents

In [ ]:
# Build your multi-agent project management system

# TODO: Create Project Manager Agent
class ProjectManagerAgent:
    def __init__(self):
        # Initialize with planning and assignment tools
        pass
    
    def plan_sprint(self, requirements: str) -> Dict:
        # Break down requirements into tasks
        pass
    
    def assign_task(self, task: Dict, agent: str) -> str:
        # Assign task to specific agent
        pass

# TODO: Create Developer Agent
class DeveloperAgent:
    def __init__(self):
        # Initialize with coding tools
        pass
    
    def write_code(self, specification: str) -> str:
        # Generate code based on specification
        pass
    
    def create_documentation(self, code: str) -> str:
        # Create technical documentation
        pass

# TODO: Create Tester Agent
class TesterAgent:
    def __init__(self):
        # Initialize with testing tools
        pass
    
    def create_test_cases(self, feature: str) -> List[str]:
        # Generate test cases
        pass
    
    def report_bug(self, test_result: Dict) -> str:
        # Create bug report
        pass

# TODO: Create Reviewer Agent
class ReviewerAgent:
    def __init__(self):
        # Initialize with review tools
        pass
    
    def review_code(self, code: str) -> Dict:
        # Review code quality
        pass
    
    def suggest_improvements(self, review: Dict) -> List[str]:
        # Suggest improvements
        pass

# TODO: Create Project Orchestrator
class ProjectOrchestrator:
    def __init__(self):
        # Initialize all agents
        # Set up communication channels
        pass
    
    def execute_project(self, project_description: str):
        # Coordinate all agents to complete project
        # 1. PM plans tasks
        # 2. Developer implements
        # 3. Tester tests
        # 4. Reviewer reviews
        # 5. Handle feedback loops
        pass

# TODO: Test your system
# Create a sample project and run it through the system

# Your code here

In [ ]:
# Solution (hidden by default)

"""
# Multi-Agent Project Management System

# Project Manager Tools
@tool
def create_user_story(feature: str) -> str:
    '''Create a user story for a feature.'''
    return f'''User Story: {feature}
    As a user, I want {feature} so that I can achieve better results.
    Acceptance Criteria:
    - Feature is implemented
    - Tests pass
    - Documentation updated'''

@tool
def estimate_effort(task: str) -> str:
    '''Estimate effort for a task in story points.'''
    # Simple estimation logic
    if "complex" in task.lower() or "architecture" in task.lower():
        return "8 story points (Large)"
    elif "medium" in task.lower() or "feature" in task.lower():
        return "5 story points (Medium)"
    else:
        return "3 story points (Small)"

# Developer Tools
@tool
def generate_code_snippet(specification: str, language: str = "python") -> str:
    '''Generate code based on specification.'''
    return f'''# {specification}
def implement_feature():
    """{specification}"""
    # Implementation here
    result = process_data()
    return result
    
# TODO: Complete implementation'''

@tool
def write_technical_doc(code: str) -> str:
    '''Write technical documentation for code.'''
    return f'''## Technical Documentation
    
### Overview
This module implements the requested feature.

### Functions
- implement_feature(): Main implementation

### Usage Example
```python
result = implement_feature()
```'''

# Tester Tools
@tool
def generate_test_cases(feature: str) -> str:
    '''Generate test cases for a feature.'''
    return f'''Test Cases for {feature}:
1. Happy Path: Feature works with valid input
2. Edge Case: Feature handles empty input
3. Error Case: Feature handles invalid input gracefully
4. Performance: Feature completes within 2 seconds'''

@tool
def run_tests(test_cases: str) -> str:
    '''Simulate running tests.'''
    import random
    if random.random() > 0.3:  # 70% pass rate
        return "✅ All tests passed (4/4)"
    else:
        return "❌ Tests failed (3/4 passed). Error Case test failed."

# Reviewer Tools
@tool
def code_quality_check(code: str) -> str:
    '''Check code quality.'''
    issues = []
    if "TODO" in code:
        issues.append("Contains TODO comments")
    if len(code.split('\n')) > 50:
        issues.append("Consider breaking into smaller functions")
    
    if issues:
        return f"Code Review Issues:\n" + "\n".join(f"- {i}" for i in issues)
    return "✅ Code quality check passed"

@tool
def suggest_refactoring(code: str) -> str:
    '''Suggest code improvements.'''
    return '''Refactoring Suggestions:
1. Extract method for complex logic
2. Add type hints for better clarity
3. Consider using design pattern (Strategy/Factory)'''

# Specialized Agents for Project Management

class ProjectManagerAgent(SpecializedAgent):
    def __init__(self):
        super().__init__(
            name="ProjectManager",
            role="Plan and coordinate project tasks",
            tools=[create_user_story, estimate_effort]
        )
        self.backlog = []
        self.sprint_tasks = []
    
    def plan_sprint(self, requirements: str) -> List[Dict]:
        # Create tasks from requirements
        tasks = [
            {"id": 1, "type": "development", "description": f"Implement {requirements}", "status": "todo"},
            {"id": 2, "type": "testing", "description": f"Test {requirements}", "status": "todo", "depends_on": 1},
            {"id": 3, "type": "review", "description": f"Review implementation", "status": "todo", "depends_on": 1},
            {"id": 4, "type": "documentation", "description": f"Update docs", "status": "todo", "depends_on": 3}
        ]
        self.sprint_tasks = tasks
        return tasks

class DeveloperAgent(SpecializedAgent):
    def __init__(self):
        super().__init__(
            name="Developer",
            role="Write code and documentation",
            tools=[generate_code_snippet, write_technical_doc]
        )
        self.completed_code = {}
    
    def implement_feature(self, task: Dict) -> Dict:
        code = self.run(f"Generate code for: {task['description']}")
        doc = self.run(f"Write documentation for the code")
        
        self.completed_code[task['id']] = {
            "code": code,
            "documentation": doc
        }
        
        return {"status": "completed", "code": code, "doc": doc}

class TesterAgent(SpecializedAgent):
    def __init__(self):
        super().__init__(
            name="Tester",
            role="Test features and report issues",
            tools=[generate_test_cases, run_tests]
        )
        self.test_results = {}
    
    def test_feature(self, task: Dict, code: str) -> Dict:
        test_cases = self.run(f"Generate test cases for: {task['description']}")
        results = self.run(f"Run tests on the implementation")
        
        self.test_results[task['id']] = {
            "test_cases": test_cases,
            "results": results,
            "passed": "passed" in results
        }
        
        return self.test_results[task['id']]

class ReviewerAgent(SpecializedAgent):
    def __init__(self):
        super().__init__(
            name="Reviewer",
            role="Review code and suggest improvements",
            tools=[code_quality_check, suggest_refactoring]
        )
        self.reviews = {}
    
    def review_code(self, task: Dict, code: str) -> Dict:
        quality = self.run(f"Check code quality")
        suggestions = self.run(f"Suggest improvements")
        
        self.reviews[task['id']] = {
            "quality_check": quality,
            "suggestions": suggestions,
            "approved": "passed" in quality
        }
        
        return self.reviews[task['id']]

# Project Orchestrator
class ProjectOrchestrator:
    def __init__(self):
        self.pm = ProjectManagerAgent()
        self.dev = DeveloperAgent()
        self.tester = TesterAgent()
        self.reviewer = ReviewerAgent()
        self.project_status = {}
    
    def execute_project(self, project_description: str) -> Dict:
        print(f"🚀 Starting Project: {project_description}\n")
        
        # Phase 1: Planning
        print("📋 Phase 1: Planning")
        tasks = self.pm.plan_sprint(project_description)
        print(f"   Created {len(tasks)} tasks\n")
        
        results = {}
        
        # Process tasks with dependencies
        for task in tasks:
            print(f"\n🔄 Processing Task {task['id']}: {task['description']}")
            
            # Check dependencies
            if task.get('depends_on'):
                dep_id = task['depends_on']
                if dep_id not in results or results[dep_id].get('status') != 'completed':
                    print(f"   ⏸️  Waiting for task {dep_id} to complete...")
                    continue
            
            # Execute based on task type
            if task['type'] == 'development':
                print("   👨‍💻 Developer working...")
                result = self.dev.implement_feature(task)
                results[task['id']] = result
                print("   ✅ Development completed")
                
            elif task['type'] == 'testing':
                print("   🧪 Tester working...")
                # Get code from previous task
                code = results[task['depends_on']].get('code', '')
                test_result = self.tester.test_feature(task, code)
                results[task['id']] = {"status": "completed", "test_result": test_result}
                
                if not test_result['passed']:
                    print("   ⚠️  Tests failed! Sending back to developer...")
                    # Feedback loop - in production, would reassign to developer
                else:
                    print("   ✅ All tests passed")
                    
            elif task['type'] == 'review':
                print("   👀 Reviewer working...")
                code = results[task['depends_on']].get('code', '')
                review = self.reviewer.review_code(task, code)
                results[task['id']] = {"status": "completed", "review": review}
                
                if review['approved']:
                    print("   ✅ Code approved")
                else:
                    print("   📝 Improvements suggested")
                    
            elif task['type'] == 'documentation':
                print("   📚 Updating documentation...")
                results[task['id']] = {"status": "completed"}
                print("   ✅ Documentation updated")
            
            task['status'] = 'completed'
        
        # Generate project summary
        completed = sum(1 for t in tasks if t['status'] == 'completed')
        
        summary = {
            "project": project_description,
            "tasks_total": len(tasks),
            "tasks_completed": completed,
            "success_rate": f"{(completed/len(tasks))*100:.0f}%",
            "results": results
        }
        
        return summary

# Test the system
print("Multi-Agent Project Management System Demo\n")
print("="*60)

orchestrator = ProjectOrchestrator()

# Execute a project
project_result = orchestrator.execute_project(
    "User authentication feature with login and registration"
)

print("\n" + "="*60)
print("\n📊 Project Summary:")
print(f"Project: {project_result['project']}")
print(f"Tasks Completed: {project_result['tasks_completed']}/{project_result['tasks_total']}")
print(f"Success Rate: {project_result['success_rate']}")

print("\n✅ Project management system demonstration complete!")
"""

print("Build a complete multi-agent project management system!")
print("The solution shows planning, development, testing, review, and coordination.")

---

## Summary and Next Steps

Congratulations! You've learned how to build autonomous AI agents. You can now:

✅ Create custom tools for agents to use
✅ Build ReAct agents that reason and act
✅ Implement OpenAI function calling
✅ Design multi-agent systems with specialization
✅ Create agent orchestrators for complex workflows
✅ Implement agent communication and collaboration

### Key Takeaways:
- **Tools Enable Capabilities**: Well-designed tools give agents real power
- **Reasoning Matters**: ReAct pattern helps agents think before acting
- **Function Calling**: OpenAI's native function calling is powerful and reliable
- **Specialization**: Different agents for different tasks improves results
- **Orchestration**: Coordinating multiple agents enables complex workflows

### Next Steps:
- **Notebook 09**: Learn about Memory and Conversations for stateful agents
- **Practice**: Build agents for your specific use cases
- **Experiment**: Try different agent architectures
- **Scale**: Implement parallel agent execution

### Additional Challenges:
1. Build an agent that can browse and extract information from websites
2. Create a code generation agent with testing capabilities
3. Implement an agent swarm for distributed problem-solving
4. Build a self-improving agent that learns from feedback
5. Create a meta-agent that can create other agents